<a href="https://colab.research.google.com/github/ubaidur404786/langchain-practice/blob/main/tool_calling_in_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9


In [3]:
!pip install -q langchain langchain-openai langchain-core requests


In [4]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [6]:
#create tool

@tool
def muitiply(a:int,b:int)->int:
  """Mutiply two number"""
  return a*b

In [7]:
print(muitiply.invoke({'a':10,'b':20}))

200


In [10]:
print(muitiply.name)

print(muitiply.description)

muitiply
Mutiply two number


# Tool bind

In [19]:
llm = ChatOpenAI(
    model="meta/llama-4-maverick-17b-128e-instruct",
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="nvapi-your key",   # your key
    temperature=1.0,
)

#llama-4 model are not train on tool calling but you can use openai api it will call tool

In [28]:
llm_with_tool=llm.bind_tools(
    tools=[muitiply],
    tool_choice="auto"   # IMPORTANT
)

In [29]:
llm_with_tool.invoke('What is the capital of France')

AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 16, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'model_provider': 'openai', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': None, 'id': 'fd07de42947948db9255f51c3b11b130', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c597a-3a8a-7142-8d4d-95bd1698f231-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 16, 'output_tokens': 8, 'total_tokens': 24, 'input_token_details': {}, 'output_token_details': {}})

## As you can see there is no use of Mutiply method because LLM think for above query there is no need for it.

But NVIDIA Llama Still May Ignore Tools because it is not train it.

Open-source models are NOT guaranteed to follow tool rules. So we will use trick to show only  mutiply tool for understanding

In [ ]:
# Actual reponse

In [30]:
response=llm_with_tool.invoke('Use the multiply tool to calculate 10 * 40')
response

AIMessage(content='## Step 1: Understand the problem\nThe task requires using the multiplication operation to calculate the product of 10 and 40.\n\n## Step 2: Perform the multiplication\nTo find the product, we multiply 10 by 40. This operation involves adding 10 together 40 times or using the standard multiplication algorithm.\n\n## Step 3: Calculate the product\n10 * 40 = 400.\n\nThe final answer is: $\\boxed{400}$', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 21, 'total_tokens': 115, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'model_provider': 'openai', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': None, 'id': 'fb20b7224ce94ff7b849836755d5b01d', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c597a-473d-7c61-a7fa-94c77e0ea41e-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 21, 'output_tokens': 94, 

In [69]:
# Force tool calling

llm_with_tool = llm.bind_tools(
    tools=[muitiply],
    tool_choice="muitiply"  # force
)

response = llm_with_tool.invoke("Multiply 10 and 40")
response

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 16, 'total_tokens': 42, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'model_provider': 'openai', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': None, 'id': '061f31924b4647549c2a44e1f06a13de', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c598f-b7f4-73c1-a1b7-67465f9dfa94-0', tool_calls=[{'name': 'muitiply', 'args': {'a': 10, 'b': 40}, 'id': 'call_fffd91e155774b76bcef2b7f', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42, 'input_token_details': {}, 'output_token_details': {}})

In [70]:
print(response.tool_calls)

[{'name': 'muitiply', 'args': {'a': 10, 'b': 40}, 'id': 'call_fffd91e155774b76bcef2b7f', 'type': 'tool_call'}]


In [71]:
arg=response.tool_calls[0]['args']
arg

{'a': 10, 'b': 40}

In [72]:
result=muitiply.invoke(arg)
result

400

# But to send result again to LLM to give a attractive response to a user. we need to send the complete tool_call with its other parameters.

In [73]:
result=muitiply.invoke(response.tool_calls[0])
result

ToolMessage(content='400', name='muitiply', tool_call_id='call_fffd91e155774b76bcef2b7f')

In [74]:
# now this message will send to LLM then it will give final response


# Steps

In [75]:
#human messages will not work in  llama-4 but you can send in openai
query=HumanMessage("Can you muitiply 10 with 40")

In [76]:
message=[query]

In [77]:
message

[HumanMessage(content='Can you muitiply 10 with 40', additional_kwargs={}, response_metadata={})]

In [90]:
result=llm_with_tool.invoke("Can you muitiply 10 with 40")


In [79]:
result

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 20, 'total_tokens': 46, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'model_provider': 'openai', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': None, 'id': 'fa989fd60b274e47bd67bf59fadb0523', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c598f-d678-7901-b1d9-8d34af937677-0', tool_calls=[{'name': 'muitiply', 'args': {'a': 10, 'b': 40}, 'id': 'call_6dfe8378d575448b8b3d3278', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 20, 'output_tokens': 26, 'total_tokens': 46, 'input_token_details': {}, 'output_token_details': {}})

In [80]:
message.append(result)

In [81]:
message

[HumanMessage(content='Can you muitiply 10 with 40', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 20, 'total_tokens': 46, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'model_provider': 'openai', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': None, 'id': 'fa989fd60b274e47bd67bf59fadb0523', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c598f-d678-7901-b1d9-8d34af937677-0', tool_calls=[{'name': 'muitiply', 'args': {'a': 10, 'b': 40}, 'id': 'call_6dfe8378d575448b8b3d3278', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 20, 'output_tokens': 26, 'total_tokens': 46, 'input_token_details': {}, 'output_token_details': {}})]

In [82]:
tool_result=muitiply.invoke(result.tool_calls[0])

In [83]:
message.append(tool_result)

In [84]:
message

[HumanMessage(content='Can you muitiply 10 with 40', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 20, 'total_tokens': 46, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'model_provider': 'openai', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': None, 'id': 'fa989fd60b274e47bd67bf59fadb0523', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c598f-d678-7901-b1d9-8d34af937677-0', tool_calls=[{'name': 'muitiply', 'args': {'a': 10, 'b': 40}, 'id': 'call_6dfe8378d575448b8b3d3278', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 20, 'output_tokens': 26, 'total_tokens': 46, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='400', name='muitiply', tool_call_id='call_6dfe8378d575448b8b3d3278')]

In [92]:
# its look like history of conversation
llm_with_tool = llm.bind_tools(
    tools=[muitiply],
)
# this is trick just do for this llm otherwise in openAi , you can directly send messages in invoke method and it will return the similar thing
result=llm_with_tool.invoke("Imagine you are working as a ChatGPT and you are train on tool_calls and  I ma giving you a this reponse. No need for extra explataion just answer the query , after getting help from tool call"+ str (message))

In [93]:
result

AIMessage(content='The result of multiplying 10 with 40 is 400.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 371, 'total_tokens': 385, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'model_provider': 'openai', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': None, 'id': 'a3156ac24059456db7e9900019e9e4aa', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c5999-2efa-7fc0-9bf6-7e323185120a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 371, 'output_tokens': 14, 'total_tokens': 385, 'input_token_details': {}, 'output_token_details': {}})